In [8]:
import pandas as pd

# Load the dataset
data = pd.read_csv('/kaggle/input/solana-historical-data-2021-2024/Solana_daily_data_2018_2024.csv')

# Display the first few rows
data.head()


time   Open   High    Low  Close      Volume
0  2021-01-01  1.510  1.865  1.495  1.847  1125773.54
1  2021-01-02  1.847  2.012  1.752  1.799  1060347.35
2  2021-01-03  1.799  2.808  2.007  2.185  1475936.63
3  2021-01-04  2.185  2.573  2.066  2.491  1543191.05
4  2021-01-05  2.491  2.634  2.145  2.158  1246011.66

In [9]:
# Basic Data Cleaning

# Rename 'time' to 'Date' for easier handling
data.rename(columns={'time': 'Date'}, inplace=True)

# Convert 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Sort the data by Date
data = data.sort_values('Date')

# Reset index after sorting
data.reset_index(drop=True, inplace=True)

# Check for missing values
print("Missing values in each column:")
print(data.isnull().sum())

# Check data types
print("\nData types:")
print(data.dtypes)

# Preview cleaned data
data.head()


Missing values in each column:
Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

Data types:
Date      datetime64[ns]
Open             float64
High             float64
Low              float64
Close            float64
Volume           float64
dtype: object


Date   Open   High    Low  Close      Volume
0 2021-01-01  1.510  1.865  1.495  1.847  1125773.54
1 2021-01-02  1.847  2.012  1.752  1.799  1060347.35
2 2021-01-03  1.799  2.808  2.007  2.185  1475936.63
3 2021-01-04  2.185  2.573  2.066  2.491  1543191.05
4 2021-01-05  2.491  2.634  2.145  2.158  1246011.66

In [10]:
# Feature Engineering

# Moving Averages
data['MA7'] = data['Close'].rolling(window=7).mean()
data['MA14'] = data['Close'].rolling(window=14).mean()
data['MA30'] = data['Close'].rolling(window=30).mean()

# Lag features
data['Lag1'] = data['Close'].shift(1)
data['Lag2'] = data['Close'].shift(2)
data['Lag3'] = data['Close'].shift(3)

# Drop rows with NA values generated by rolling and lag features
data = data.dropna()

# Preview the new features
data.head()


Date   Open   High    Low  Close      Volume       MA7      MA14  \
29 2021-01-30  3.839  4.325  3.789  4.218   771726.59  3.851429  3.699714   
30 2021-01-31  4.218  4.792  4.167  4.263  1439455.37  3.939857  3.731071   
31 2021-02-01  4.263  4.616  4.094  4.616   437158.49  4.070714  3.789500   
32 2021-02-02  4.616  5.393  4.476  5.255  1033097.85  4.245571  3.905214   
33 2021-02-03  5.255  5.783  5.077  5.736  1034840.35  4.537429  4.048071   

        MA30   Lag1   Lag2   Lag3  
29  3.238667  3.827  3.847  3.693  
30  3.319200  4.218  3.827  3.847  
31  3.413100  4.263  4.218  3.827  
32  3.515433  4.616  4.263  4.218  
33  3.623600  5.255  4.616  4.263

In [11]:
# Train/Test Split

# Define input features and target variable
features = ['Lag1', 'Lag2', 'Lag3', 'MA7', 'MA14', 'MA30']
target = 'Close'

X = data[features]
y = data[target]

# Split data: 80% train, 20% test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

print(f"Train size: {X_train.shape[0]} rows")
print(f"Test size: {X_test.shape[0]} rows")


Train size: 1071 rows
Test size: 268 rows


In [12]:
# Model Training

# Linear Regression
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# XGBoost Regressor
from xgboost import XGBRegressor

xgb_model = XGBRegressor(random_state=42, verbosity=0)
xgb_model.fit(X_train, y_train)

print("✅ All three models trained successfully!")


✅ All three models trained successfully!


In [13]:
# Model Evaluation

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Function to evaluate and print metrics
def evaluate_model(name, model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"📈 Model: {name}")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2 Score: {r2:.4f}")
    print("-" * 30)

# Evaluate all models
evaluate_model("Linear Regression", lr_model, X_test, y_test)
evaluate_model("Random Forest", rf_model, X_test, y_test)
evaluate_model("XGBoost", xgb_model, X_test, y_test)


📈 Model: Linear Regression
MAE: 4.7484
RMSE: 6.2460
R2 Score: 0.9473
------------------------------
📈 Model: Random Forest
MAE: 5.7456
RMSE: 7.2469
R2 Score: 0.9291
------------------------------
📈 Model: XGBoost
MAE: 5.9953
RMSE: 7.7185
R2 Score: 0.9196
------------------------------


In [14]:
# Save all models

import joblib

# Save Linear Regression model
joblib.dump(lr_model, '/kaggle/working/linear_regression_model.pkl')

# Save Random Forest model
joblib.dump(rf_model, '/kaggle/working/random_forest_model.pkl')

# Save XGBoost model
joblib.dump(xgb_model, '/kaggle/working/xgboost_model.pkl')

print("✅ All three models saved successfully!")


✅ All three models saved successfully!


In [15]:
# Load Solana Hourly Data

import pandas as pd

# Load the uploaded CSV file
hourly_data = pd.read_csv('/kaggle/input/solana-2024-25-hourly/solana_hourly.csv')

# Display the first few rows
hourly_data.head()


Open Time    Open    High     Low   Close     Volume
0  2024-01-01 00:00:00  101.72  102.79  101.56  101.96  196680.93
1  2024-01-01 01:00:00  101.96  104.75  101.51  104.12  242095.33
2  2024-01-01 02:00:00  104.11  104.34  103.22  103.69  125943.27
3  2024-01-01 03:00:00  103.69  103.99  102.54  103.07  126749.20
4  2024-01-01 04:00:00  103.07  103.98  101.53  102.62  157782.35

In [16]:
#  Preprocessing and Feature Engineering for Hourly Data

# 1. Make sure 'Open Time' is datetime
hourly_data['Open Time'] = pd.to_datetime(hourly_data['Open Time'])

# 2. Sort by Open Time (important for time series)
hourly_data = hourly_data.sort_values('Open Time').reset_index(drop=True)

# 3. Create Moving Averages
hourly_data['MA3'] = hourly_data['Close'].rolling(window=3).mean()
hourly_data['MA6'] = hourly_data['Close'].rolling(window=6).mean()
hourly_data['MA12'] = hourly_data['Close'].rolling(window=12).mean()

# 4. Create Lag Features
hourly_data['Lag1'] = hourly_data['Close'].shift(1)
hourly_data['Lag2'] = hourly_data['Close'].shift(2)
hourly_data['Lag3'] = hourly_data['Close'].shift(3)

# 5. Drop rows with any missing values (from rolling and shifting)
hourly_data = hourly_data.dropna()

# 6. Preview the processed data
hourly_data.head()


Open Time    Open    High     Low   Close     Volume         MA3  \
11 2024-01-01 11:00:00  104.14  104.59  103.74  104.32  132444.07  104.163333   
12 2024-01-01 12:00:00  104.33  106.10  103.86  104.76  267177.81  104.406667   
13 2024-01-01 13:00:00  104.76  105.81  104.36  105.31  191274.15  104.796667   
14 2024-01-01 14:00:00  105.31  105.34  104.28  104.39  136742.53  104.820000   
15 2024-01-01 15:00:00  104.39  106.39  104.15  105.98  183898.00  105.226667   

           MA6        MA12    Lag1    Lag2    Lag3  
11  103.303333  103.076667  104.14  104.03  102.48  
12  103.706667  103.310000  104.32  104.14  104.03  
13  104.173333  103.409167  104.76  104.32  104.14  
14  104.491667  103.467500  105.31  104.76  104.32  
15  104.816667  103.710000  104.39  105.31  104.76

In [17]:
# Train-Test Split and Model Retraining

from sklearn.model_selection import train_test_split

# 1. Define Features and Target
features = ['Lag1', 'Lag2', 'Lag3', 'MA3', 'MA6', 'MA12']
target = 'Close'

X = hourly_data[features]
y = hourly_data[target]

# 2. Split into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

print(f"Train size: {X_train.shape[0]} rows")
print(f"Test size: {X_test.shape[0]} rows")

# 3. Retrain Models

# Linear Regression
from sklearn.linear_model import LinearRegression
lr_model_hourly = LinearRegression()
lr_model_hourly.fit(X_train, y_train)

# Random Forest
from sklearn.ensemble import RandomForestRegressor
rf_model_hourly = RandomForestRegressor(random_state=42)
rf_model_hourly.fit(X_train, y_train)

# XGBoost
from xgboost import XGBRegressor
xgb_model_hourly = XGBRegressor(random_state=42, verbosity=0)
xgb_model_hourly.fit(X_train, y_train)

print("✅ Models retrained on hourly data!")


Train size: 9284 rows
Test size: 2322 rows
✅ Models retrained on hourly data!


In [18]:
# Evaluate Retrained Models on Hourly Data

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Function to evaluate a model
def evaluate_model(name, model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"📈 Model: {name}")
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R2 Score: {r2:.4f}")
    print("-" * 30)

# Evaluate all models
evaluate_model("Linear Regression (Hourly)", lr_model_hourly, X_test, y_test)
evaluate_model("Random Forest (Hourly)", rf_model_hourly, X_test, y_test)
evaluate_model("XGBoost (Hourly)", xgb_model_hourly, X_test, y_test)


📈 Model: Linear Regression (Hourly)
MAE: 0.0000
RMSE: 0.0000
R2 Score: 1.0000
------------------------------
📈 Model: Random Forest (Hourly)
MAE: 0.6419
RMSE: 1.0386
R2 Score: 0.9993
------------------------------
📈 Model: XGBoost (Hourly)
MAE: 0.8776
RMSE: 1.4202
R2 Score: 0.9987
------------------------------


In [19]:
# Step: Save the best model (Linear Regression Hourly)

import joblib

# Save model
joblib.dump(lr_model_hourly, '/kaggle/working/final_hourly_model.pkl')

print("✅ final_hourly_model.pkl saved successfully!")


✅ final_hourly_model.pkl saved successfully!
